In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Crawl data from Borowitz Report (The Newyorker)

In [10]:
headline = []
article_link = []
is_sarcastic = []
date = []

title_news=["news","humor"]
link = 'https://www.newyorker.com/latest/{}/page/{}' #Đường dẫn đến trang web theo page

for news in title_news:
  for i in range(400):
    req = requests.get(link.format(news,i)) #Tạo request
    soup = BeautifulSoup(req.text, "lxml")
    g = soup.find_all('div',class_='River__riverItemContent___2hXMG') #Tìm tất cả thẻ chứa thông tin bài báo

    ex= g[0]
    year= int(ex.find_all('h6',class_="River__publishDate___1fSSK")[0].get_text().split(', ')[-1])

    if year <2018:
      break

    for i in g:
      headline.append(i.find('a',class_='Link__link___3dWao ').get_text()) #Lấy tiêu đề
      url = 'https://www.newyorker.com/' + i.find('a',class_='Link__link___3dWao ').get('href') #Lấy URL
      article_link.append(url)
      date.append(i.find('h6').get_text()) #Lấy ngày phát hành
      is_sarcastic.append(1) #Lable

In [11]:
df = pd.DataFrame({'headline':headline,'article_link':article_link,'is_sarcastic':is_sarcastic,'date':date})
df

,headline,article_link,is_sarcastic,date
0,The New Yorker Wins Six National Magazine Awards,https://www.newyorker.com//news/news-desk/the-...,1,"June 10, 2021"
1,An Ode to the No-Hitter,https://www.newyorker.com//sports/sporting-sce...,1,"June 7, 2021"
2,"The Ridiculous, Not-So-Ridiculous Spectacle of...",https://www.newyorker.com//sports/sporting-sce...,1,"June 5, 2021"
3,What if Pro Sports Leagues Were Controlled by ...,https://www.newyorker.com//sports/sporting-sce...,1,"June 3, 2021"
4,"Knicks Fans Are Together Again, for Better and...",https://www.newyorker.com//culture/dept-of-ret...,1,"June 2, 2021"
...,...,...,...,...
4415,Paul Ryan Says He Will Retire Once He Has Wrec...,https://www.newyorker.com//humor/borowitz-repo...,1,"January 2, 2018"
4416,"Daily Cartoon: Tuesday, January 2nd",https://www.newyorker.com//cartoons/daily-cart...,1,"January 2, 2018"
4417,Retirement Planning vs. Quest for the Amulet o...,https://www.newyorker.com//humor/daily-shouts/...,1,"January 2, 2018"
4418,Mistakes You’re Going to Keep Making Forever,https://www.newyorker.com//humor/daily-shouts/...,1,"January 1, 2018"


In [12]:
df.to_csv('newyorker.csv',index=False)

# Crawl data from Euronews

Trang báo Euronews cho phép xem các báo phát hành theo ngày (ví dụ với đường dẫn https://www.euronews.com/2021/06/05 ta có thể xem tất cả các bài báo đã phát hành vào ngày 05/06/2021)nên ta có thể dễ dàng crawl tiêu đề bài báo ở một ngày cụ thể với url dạng https://www.euronews.com/yyyy/mm/dd)

In [4]:
from datetime import date,timedelta
today = date.today()
d = today.strftime("%Y/%m/%d")
d

'2021/06/11'

In [ ]:
headline = []
article_link = []
is_sarcastic = []
date = []
link = 'https://www.euronews.com/' # Đường dẫn đến trang web
while(today.year>=2019):
  req = requests.get(link+d) # d là yyyy/mm/dd
  print(req)
  if(req.status_code==200): # Kiểm tra request thành công hay không?
    soup = BeautifulSoup(req.text, "lxml")

    for i in soup.find_all('a',class_ = 'm-object__title__link '):
      headline.append(i.get('title')) #Lấy tiêu đề 
      article_link.append('https://www.euronews.com/'+i.get('href')) #Lấy URL
      is_sarcastic.append(0)
      date.append(d)
  
  today -= timedelta(1)
  d = today.strftime("%Y/%m/%d")


In [6]:
df = pd.DataFrame({'headline':headline,'article_link':article_link,'is_sarcastic':is_sarcastic,'date':date})
df

,headline,article_link,is_sarcastic,date
0,China passes law to counter foreign sanctions,https://www.euronews.com//2021/06/11/china-pas...,0,2021/06/11
1,Jill Biden offers 'love' from America on her j...,https://www.euronews.com//2021/06/11/jill-bide...,0,2021/06/11
2,Joe Biden will be the 13th US president Queen ...,https://www.euronews.com//2021/06/11/joe-biden...,0,2021/06/11
3,China releases images taken by its space rover...,https://www.euronews.com//2021/06/11/china-rel...,0,2021/06/11
4,Matic delivers Croatia a first ever World Cham...,https://www.euronews.com//2021/06/10/matic-del...,0,2021/06/10
...,...,...,...,...
23999,Parched Jordan to drill a half-mile undergroun...,https://www.euronews.com//2019/01/01/jordan-dr...,0,2019/01/01
24000,"In 2019, the nuclear deal is hanging by a thre...",https://www.euronews.com//2019/01/01/the-iran-...,0,2019/01/01
24001,Feeling lonely? You're not alone. How to find ...,https://www.euronews.com//2019/01/01/lonely-yo...,0,2019/01/01
24002,France: Eight people spend New Year's Eve stuc...,https://www.euronews.com//2019/01/01/france-ei...,0,2019/01/01


In [7]:
df.to_csv('euronews.csv')

#crawl data from National Post

In [17]:
import requests
from bs4 import BeautifulSoup
from datetime import date , timedelta
from tqdm import tqdm
import time
import pandas as pd

Trang báo National Post cho phép xem các bài báo theo nội dung tìm kiếm

Mỗi lần tìm kiếm sẽ được một list 10 bài báo và ta sẽ duyệt qua các list bài báo tìm kiếm được để thu thập các tiêu đề và đường link bài báo

In [18]:
link5 = "https://nationalpost.com/search/?search_text=news&date_range=-7300d&sort=desc&from="

In [19]:
def get_data5(link5):
  response = requests.get(link5)
  lst = []
  soup = BeautifulSoup(response.content, "html.parser")
  table  = soup.findAll('div', class_="article-card__content") #Tìm list bài báo
  times = []
  for i in table: #tìm thẻ span chứa ngày tháng năm phát hành bài báo
    time = []
    z = i.find('div') 
    for j in z:
      k = j.find('span')
      time.append(k)
    year = time[3].text[-5:]
    try :
      if (year == ' ago '):
        times.append(2021)
      else:
        times.append(int(year))
    except:
      times.append(2018)
  links = [link.find('a').attrs["href"] for link in table] #tìm các đường link bài báo
  headline = []    #find headline
  for link in table: #Tìm tiêu đề bài báo
    text = link.find('a').text.split('   ') 
    headline.append(text[0])
  for i in range(len(links)):
    dic = dict()
    dic["article_link"] = "https://nationalpost.com/"+ links[i]
    dic["headline"] = headline[i]
    dic["is_sarcastic"] = 0
    print(dic)
    if (int(times[i])> 2018): #thêm nhưng bài báo phát hành sau năm 2018
      lst.append(dic)
  return lst  

In [ ]:
lst = []
for index in tqdm(range(0, 10000,10)): #tìm kiếm 10000 bài báo
  link = link5 + str(index)
  lst += get_data5(link)

In [21]:
def extra_csv(lst):
  headline = []
  article_link = []
  is_sarcastic = []
  for i in lst:
    article_link.append(i['article_link'])
    headline.append(i['headline'])
    is_sarcastic.append(i['is_sarcastic'])
  df = pd.DataFrame({'headline':headline,'article_link':article_link,'is_sarcastic':is_sarcastic})
  return df
df =extra_csv(lst)

In [22]:
df.head()

,headline,article_link,is_sarcastic
0,"Red Sox host Vladimir Guerrero Jr., Blue Jays",https://nationalpost.com//pmn/sports-pmn/red-s...,0
1,Philippines to give vaccinated elderly more ...,https://nationalpost.com//pmn/health-pmn/phili...,0
2,MLB roundup: Twins overtake Yanks with 4-run...,https://nationalpost.com//pmn/sports-pmn/mlb-r...,0
3,"Daily horoscope for Friday, June 11, 2021",https://nationalpost.com//life/horoscopes/dail...,0
4,"Lucas Giolito, White Sox aim to keep Tigers ...",https://nationalpost.com//pmn/sports-pmn/lucas...,0


In [23]:
df.to_csv('TheTrueNorthTime.csv') # lưu thành file csv

In [24]:
df

,headline,article_link,is_sarcastic
0,"Red Sox host Vladimir Guerrero Jr., Blue Jays",https://nationalpost.com//pmn/sports-pmn/red-s...,0
1,Philippines to give vaccinated elderly more ...,https://nationalpost.com//pmn/health-pmn/phili...,0
2,MLB roundup: Twins overtake Yanks with 4-run...,https://nationalpost.com//pmn/sports-pmn/mlb-r...,0
3,"Daily horoscope for Friday, June 11, 2021",https://nationalpost.com//life/horoscopes/dail...,0
4,"Lucas Giolito, White Sox aim to keep Tigers ...",https://nationalpost.com//pmn/sports-pmn/lucas...,0
...,...,...,...
9995,Haiti government denounces plot to oust pres...,https://nationalpost.com//pmn/news-pmn/crime-p...,0
9996,Biden says challenging for U.S. to reach her...,https://nationalpost.com//pmn/health-pmn/biden...,0
9997,"Quebec passes 10,000 COVID-19 deaths as Onta...",https://nationalpost.com//pmn/news-pmn/canada-...,0
9998,Haiti prime minister says about 20 people de...,https://nationalpost.com//pmn/news-pmn/crime-p...,0


# citynews

In [ ]:
def get_info(x):
  filter1 =x.find('a',class_='title')
  filter2 =x.find('span')

  tilte= filter1.get_text()
  link=  filter1.get('href')
  date=  filter2.get_text()
  return [tilte,link,0,date]

In [ ]:
url="https://toronto.citynews.ca/category/national/page/{}/"

data=[]
for i in range(2500):
  page= requests.request('GET',url.format(i))
  soup= bs(page.text)
  list_news=soup.find_all('div',class_='post_row col-lg-16 col-md-16 col-sm-16 col-xs-16')

  date= list_news[0].find('span').get_text()
  year= date.split(', ')[1].split(' ')[0]
  
  print(date)
  if int(year) <2018:
    break
  
  for x in list_news:
    temp= get_info(x)
    if temp is not None:
      data.append(temp)


In [ ]:
len(data)

30780

In [ ]:

df_kha3=pd.DataFrame(data=data,columns=['headline','article_link','is_sarcastic','date'])
df_kha3

,headline,article_link,is_sarcastic,date
0,Has TV reached the end of the ‘binge watch’ era?,https://toronto.citynews.ca/2021/06/11/has-tv-...,0,"June 11, 2021 5:12 AM"
1,Reopening worries and wake of sorrow: In The N...,https://toronto.citynews.ca/2021/06/11/reopeni...,0,"June 11, 2021 4:30 AM"
2,Canada still mum on vaccine-sharing details as...,https://toronto.citynews.ca/2021/06/11/canada-...,0,"June 11, 2021 4:00 AM"
3,Indigenous monitor role appointment faces priv...,https://toronto.citynews.ca/2021/06/11/indigen...,0,"June 11, 2021 4:00 AM"
4,Feds face calls for inquiry into military’s ha...,https://toronto.citynews.ca/2021/06/11/feds-fa...,0,"June 11, 2021 4:00 AM"
...,...,...,...,...
30775,Power surge traps 120 on B.C. ski lift; all sa...,https://toronto.citynews.ca/2017/12/31/power-s...,0,"December 31, 2017 10:25 PM"
30776,Toronto Polar Bear Club’s New Year’s Day dip c...,https://toronto.citynews.ca/2017/12/31/toronto...,0,"December 31, 2017 9:08 PM"
30777,"As temperatures drop and shelters fill up, con...",https://toronto.citynews.ca/2017/12/31/as-temp...,0,"December 31, 2017 6:53 PM"
30778,Trudeau calls on Canadians to work together in...,https://toronto.citynews.ca/2017/12/31/trudeau...,0,"December 31, 2017 5:17 PM"


In [ ]:
df_kha3.to_csv('citynews.csv')

#crawl data from beveaton

In [25]:
def get_news_from_div1(raw):
    
    try:
        
        #get title

        title= raw.find_all('h3')[0].a.contents[0]

        #get date
        date= raw.find_all('time',class_='time')[0]['content'].split('T')[0]

        #get link
        link= raw.find_all('h3')[0].a.get('href')

        return [title,link,1,date]
    except :
        return None


In [26]:
def get_raw_list_news1(url,year_stop):

    try:
        page= requests.request('GET',url)
        soup= bs(page.text)

        if page.status_code !=200: 
          return -2,[]

        


        #div section
        filter1=soup.body.find_all('div')[2].find_all('section')[0] 
        
        #div cf
        filter2= filter1.find_all('div',class_="cf")[-10]

        #div "row mb..." has list div of news
        filter1=soup.find_all('div',class_="posts border ajaxify-pagination")[0]

        # listing news div
        list_news=filter1.find_all('article')

        ex= list_news[0]
        year=ex.find_all('time',class_='time')[0]['content']

        year=int(year.split('-')[0])
        print(year)
        if year < year_stop:
            return -1,[]

        return page.status_code,list_news
    except ValueError:
        return -2,[]

In [ ]:
url='https://www.thebeaverton.com/page/{}/'

count_failed=0
data1=[] 
for i in range(600):
  response , ls_news= get_raw_list_news1(url.format(i),2018)

  if response==-1: 
      print(1)
      break
  if response==400:
      print(2)
      count_failed+=1
      continue
  if len(ls_news)!=0:
      #print(3)
      for x in ls_news:
          temp= get_news_from_div1(x)
          if temp is not None:
              data1.append(temp)



In [ ]:
data1

In [ ]:

df=pd.DataFrame(data=data1,columns=['article_link','headline','date','is_sarcastic'])
df

In [ ]:
df.to_csv('beaverton.csv')

#crawl data from babylon

In [13]:
import requests
from bs4 import BeautifulSoup as bs
import html
import pandas as pd


In [14]:
def get_raw_list_news(url,year_stop):

    try:
        page= requests.request('GET',url)
        text= page.text

      

        #div body
        filter1= text.split('<body class="">')[1]
        filter1= filter1.split('</body>')[0]
        
        #div main
        filter2= filter1.split('main class="py-4"')[1]
        filter2= filter2.split('</main>')[0]

        #div "row mb..." has list div of news
        filter3= filter2.split('<div class="row mb-2 gutter-4">')[1]
        filter3= filter3.split('<ul class="pagination" role="navigation">')[0]

        # listing news div
        list_news= filter3.split('<div class="col-sm-6">')

        year=int(get_news_from_div(list_news[2],url)[-1].split(',')[-1])
        print(year)
        if year < year_stop:
            return -1,[]

        return page.status_code,list_news
    except ValueError:
        return -2,[]

In [15]:
def get_news_from_div(raw,url):
    
    try:
        url= url.split('/news')[0]
        #get title
        title= raw.split(':title=')[1].split(':image')[0]
        title= html.unescape(title)[2:-3]

        #get date
        date=raw.split(':published_on=')[1].split(':primary_category')[0][2:-3]

        #get link
        link= raw.split(':path=')[1].split(':is_premium=')[0][2:-3]
        return [title,url+link,1,date ]
    except :
        return None


In [ ]:
url='https://babylonbee.com/news?page={}'

count_failed=0
data=[] 
for i in range(350):
  response , ls_news= get_raw_list_news(url.format(i),2018)

  if response==-1: 
      print(1)
      break
  if response==400:
      print(2)
      count_failed+=1
      continue
  if len(ls_news)!=0:
      #print(3)
      for x in ls_news:
          temp= get_news_from_div(x,url)
          if temp is not None:
              data.append(temp)



In [ ]:
len(data)

In [ ]:

df=pd.DataFrame(data=data,columns=['headline','article_link',,'is_sarcastic','date'])
df

In [ ]:
df

In [ ]:
df.to_csv('babylon.csv')

# Merge csv

read csv

In [65]:
df_nghia1= pd.read_csv('euronews.csv',index_col=0)

In [64]:
df_nghia2= pd.read_csv('newyorker.csv')

In [71]:
df_kha1= pd.read_csv('beaverton.csv',index_col=0)

In [14]:
df_kha2= pd.read_csv('babylon.csv',index_col=0)

In [15]:
df_lam = pd.read_csv('TheTrueNorthTime.csv',index_col=0)

In [83]:
df_kha3= pd.read_csv('citynews.csv',index_col=0)

merge and save

In [48]:
df= pd.DataFrame()

In [68]:
df = df_kha2.append((df_kha3,df_kha1,df_nghia1,df_nghia2,df_lam))

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78234 entries, 0 to 30779
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      78234 non-null  object
 1   article_link  78234 non-null  object
 2   is_sarcastic  78234 non-null  int64 
 3   date          68234 non-null  object
dtypes: int64(1), object(3)
memory usage: 3.0+ MB


In [88]:
df.to_csv('dataset-news-is_sarcastic.csv')